# Marbled Salamander Occurrence Records Summary
This notebook is a tool for exploring data sets requested from GBIF (and eventually other sources), and mostly for developing criteria for filtering records (filter sets).  When the entire notebook is run, it retrieves records according to the filter sets specified and saves the results (records and some summary tables) in an sqlite database.  Some information is pulled from the parameters.sqlite database that is saved in this repository.

### General Setup

In [1]:
%matplotlib inline
import sqlite3
import pprint
import json
import pandas as pd
import numpy as np
#import geopandas as gpd
pd.set_option('display.width', 600)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_rows', 150)
from IPython.display import Image
from pygbif import occurrences
import matplotlib.pyplot as plt
import os
from datetime import datetime
t1 = datetime.now()
import sys
sys.path.append("T:/GAP/Data")
import gapconfig as config

There is a bug with mpl_toolkits, the following code is a temp fix, hopefully.
https://stackoverflow.com/questions/52911232/basemap-library-using-anaconda-jupyter-notebooks-keyerror-proj-lib/54087410#54087410

In [2]:
os.environ['PROJ_LIB'] = r'c:\Users\nmtarr\AppData\Local\Continuum\miniconda3\envs\occurrences\Library\share'

### Set Variables
Fill out stuff in this cell.  

Notes:
default_coordUncertainty -- coordinateUncertaintyInMeters is often not provided.  Here is an option to use a default.  If you don't want anything entered, set this equal to False (boolean, not string).

In [3]:
species_id = 'amasax0'
summary_name = 'Marble'
gbif_req_id = 'GBIFr24'
gbif_filter_id = 'GBIFf9'
workDir = 'T:/temp/'
codeDir = 'T:/code/wildlife-wrangler/'
sys.path.append(codeDir)
import repo_functions as functions
inDir = workDir + 'Inputs/'
outDir = workDir + 'Outputs/'
default_coordUncertainty = 1000 # Note above.
SRID_dict = {'WGS84': 4326, 'AlbersNAD83': 102008} # Used in file names for output.
spdb = outDir + species_id + gbif_req_id + gbif_filter_id + '.sqlite'

username = config.gbif_username
password = config.gbif_password
email = config.gbif_email

print("Notebook run " + str(t1))
print(spdb)

Notebook run 2020-03-13 15:36:45.897177
T:/temp/Outputs/amasax0GBIFr24GBIFf9.sqlite


In [4]:
connjup = sqlite3.connect(codeDir + 'parameters.sqlite')
cursorjup = connjup.cursor()

# Get some variables
years = connjup.execute("""SELECT years_range 
                           FROM gbif_requests WHERE request_id = '{0}'""".format(gbif_req_id)).fetchone()[0]
gap_id = connjup.execute("""SELECT gap_id
                            FROM species_concepts WHERE species_id = '{0}'""".format(species_id)).fetchone()[0]
common_name = connjup.execute("""SELECT common_name
                                 FROM species_concepts WHERE species_id = '{0}'""".format(species_id)).fetchone()[0]

# Species Concept
Display information on the species from the parameters.sqlite database.

In [5]:
vals = cursorjup.execute("SELECT * FROM species_concepts WHERE species_id = '{0}';".format(species_id)).fetchall()[0]
cols = [x[1] for x in cursorjup.execute("PRAGMA table_info('species_concepts')").fetchall()]
pprint.pprint(dict(zip(cols, vals)))

{'bcb_id': None,
 'breeding_months': None,
 'common_name': 'marbled salamander',
 'detection_distance_meters': 0,
 'ebird_id': None,
 'end_year': None,
 'error_tolerance': 20,
 'fws_id': None,
 'gap_id': 'amasax',
 'gbif_id': '2431951',
 'geometry': None,
 'itis_tsn': '173591',
 'lumped_into': None,
 'migratory': '0',
 'notes': None,
 'pad': 1,
 'scientific_name': 'Ambystoma opacum',
 'species_id': 'amasax0',
 'split_from': None,
 'start_year': '',
 'vetted_date': '3/10/2020',
 'vetted_how': 'Consulted ITIS and NatureServe.',
 'vetted_who': 'N. Tarr',
 'wintering_months': None}


# Filters
Display the parameters of the request filter set.  These are deployed during the step where records are retrieved from the API.

In [6]:
df1 = pd.read_sql_query(sql="SELECT * FROM gbif_requests WHERE request_id = '{0}'".format(gbif_req_id), con=connjup)
print("THE REQUEST FILTER SET")
print(df1.loc[0])

THE REQUEST FILTER SET
request_id                            GBIFr24
source                                   GBIF
lat_range                                None
lon_range                                None
years_range                         1999,2020
months_range                             1,12
geoissue                                False
coordinate                               True
country                                    US
geometry                                 None
creator                               N. Tarr
notes           Used for tests of large do...
Name: 0, dtype: object


Display the parameters of the post-request filter set.  These are deployed after the records are retrieved from the API, but before they are stored in the occurrence record sqlite database.

In [7]:
df2 = pd.read_sql_query(sql="SELECT * FROM gbif_filters WHERE filter_id = '{0}'".format(gbif_filter_id), con=connjup)
print("THE POST REQUEST FILTER SET")
print(df2.loc[0])

THE POST REQUEST FILTER SET
filter_id                                            GBIFf9
dataset                                                GBIF
institutions_omit                                          
collection_codes_omit                                      
datasets_omit                                          None
has_coordinate_uncertainty                                0
max_coordinate_uncertainty                            10000
bases_omit                                                 
sampling_protocols_omit                                    
issues_omit                                            None
duplicates_OK                                          True
creator                                             N. Tarr
notes                         Used for speed tests.  Let...
Name: 0, dtype: object


## Filter set justification
**default_coord_uncertainty:** 

**years_range:**

**months_range:** 

**geoissue:** 

**coordinate:** 

**country:**

**geometry:**

**collection_codes_omit:** 

**institutions_omit:** 

**has_coordinate_uncertainty:** 

**max_coordinate_uncertainty:** 

**bases_omit:** 

**protocols_omit:**

**sampling_protocols_omit:** 

**issues_omit:**

**duplicates:**

In [8]:
functions.retrieve_gbif_occurrences(codeDir, species_id, inDir, spdb, gbif_req_id, gbif_filter_id, 
                                    default_coordUncertainty, outDir, summary_name, username,
                                   password, email)

SPATIALITE_SECURITY set to relaxed
Created occurrence db: 0:00:00.248563
Got request params and sorted out geometry constraints: 0:00:00.001000
3675 records available
Downloaded records: 0:00:17.250182
	3675 records exist with the request parameters
Empty DataFrame
Columns: [occ_id, institutionCode, collectionCode, datasetName]
Index: []
          occ_id institutionCode collectionCode                    datasetName
0     2549996578     iNaturalist   Observations  iNaturalist research-grade...
1     2550032730     iNaturalist   Observations  iNaturalist research-grade...
2     2550033744     iNaturalist   Observations  iNaturalist research-grade...
3     2550043044     iNaturalist   Observations  iNaturalist research-grade...
4     2550058631     iNaturalist   Observations  iNaturalist research-grade...
...          ...             ...            ...                            ...
3670  1852456898            CUMV          Herps                        UNKNOWN
3671  1852457901            

### How many records made it through the filters?
This is the number that was actually saved in the occurrence record sqlite database.

In [9]:
conn_occ= sqlite3.connect(spdb)
curs_occ = conn_occ.cursor()
record_count = curs_occ.execute("SELECT COUNT(occ_id) FROM occurrences WHERE species_id = '{0}'".format(species_id)).fetchone()
print(str(record_count[0]) + " records")

3270 records


### Are there duplicate records left?
Duplicates based on latitude, longitude, and date-time should of been removed, with the record with the highest individualCount retained.

In [10]:
dups0 = curs_occ.execute("""SELECT count(occ_id) FROM occurrences WHERE occ_id NOT IN (SELECT occ_id FROM occurrences GROUP BY latitude, longitude, occurrenceDate HAVING max(IndividualCount));""").fetchall()
print(str(dups0[0][0]) + ' duplicate records retained based on xy coordinate and date-time')

410 duplicate records retained based on xy coordinate and date-time


# Sources
#### Pre-filters

In [11]:
tables = cursorjup.execute("SELECT table_name FROM table_descriptions").fetchall()
tables = [x[0] for x in tables]

filter_sets = [gbif_req_id, gbif_filter_id]

sources = []
for s in filter_sets:
    s = s.strip()
    for tab in tables:
        columns = cursorjup.execute("SELECT column_name FROM column_descriptions WHERE table_name = '{0}'".format(tab)).fetchall()
        columns = [x[0] for x in columns]
        for col in columns:
            try:
                a = cursorjup.execute("SELECT source FROM {1} WHERE {2} = '{0}'".format(s, tab, col)).fetchone()[0]
                sources.append(a)
            except:
                pass
print(list(set(sources))[0])

GBIF


In [12]:
sources = pd.read_sql(sql="SELECT * FROM pre_filter_source_counts;", con=conn_occ)
print(sources)

   institutionCode         collectionCode                    datasetName     0
0              AUM                  Herps                        UNKNOWN    13
1               CM                  Herps                        UNKNOWN     1
2             CUMV                  Herps                        UNKNOWN   160
3             FHSM                   HERP                        UNKNOWN     6
4              GSU               GSU-Herp  Georgia Southern Universit...     6
5            LSUMZ                  Herps                        UNKNOWN     1
6              MCZ                   Herp                        UNKNOWN   187
7              MCZ                HerpOBS                        UNKNOWN     1
8              MPM                      H                        UNKNOWN     8
9             NCSM                   Herp    NCSM Herpetology Collection    59
10            NEON              HEVC-GBTS                        UNKNOWN     1
11            OMNH             Amphibians           

#### Post-filters

In [13]:
tables = cursorjup.execute("SELECT table_name FROM table_descriptions").fetchall()
tables = [x[0] for x in tables]

filter_sets = [gbif_req_id, gbif_filter_id]

sources = []
for s in filter_sets:
    s = s.strip()
    for tab in tables:
        columns = cursorjup.execute("SELECT column_name FROM column_descriptions WHERE table_name = '{0}'".format(tab)).fetchall()
        columns = [x[0] for x in columns]
        for col in columns:
            try:
                a = cursorjup.execute("SELECT source FROM {1} WHERE {2} = '{0}'".format(s, tab, col)).fetchone()[0]
                sources.append(a)
            except:
                pass
print(list(set(sources))[0])

sql = "SELECT institutionCode, collectionCode, datasetName, COUNT(occ_id) FROM occurrences GROUP BY institutionCode, collectionCode, datasetName;"
sources = pd.read_sql(sql=sql, con=conn_occ)
print(sources)

GBIF


DatabaseError: Execution failed on sql 'SELECT institutionCode, collectionCode, datasetName, COUNT(occ_id) FROM occurrences GROUP BY institutionCode, collectionCode, datasetName;': no such column: institutionCode

# Bases
#### Pre-filter

In [14]:
bases = pd.read_sql(sql="SELECT value as basisOfRecord, count FROM pre_filter_value_counts WHERE attribute = 'bases';", con=conn_occ)
print(bases)

         basisOfRecord  count
0    HUMAN_OBSERVATION   2499
1   PRESERVED_SPECIMEN   1175
2  MACHINE_OBSERVATION      1


#### Post-filters

In [15]:
sql = "SELECT basisOfRecord, COUNT(occ_id) as count FROM occurrences GROUP BY basisOfRecord;"
bases = pd.read_sql(sql=sql, con=conn_occ)
print(bases)

DatabaseError: Execution failed on sql 'SELECT basisOfRecord, COUNT(occ_id) as count FROM occurrences GROUP BY basisOfRecord;': no such column: basisOfRecord

# Protocols
#### Pre-filters

In [16]:
protocol = pd.read_sql(sql="SELECT value as samplingProtocol, count FROM pre_filter_value_counts WHERE attribute = 'samplingProtocols';", con=conn_occ)
print(protocol)

  samplingProtocol  count
0           UKNOWN   3675


#### Post-filters

In [17]:
sql = "SELECT samplingProtocol, COUNT(occ_id) as count FROM occurrences GROUP BY samplingProtocol;"
print(pd.read_sql(sql=sql, con=conn_occ))

DatabaseError: Execution failed on sql 'SELECT samplingProtocol, COUNT(occ_id) as count FROM occurrences GROUP BY samplingProtocol;': no such column: samplingProtocol

# Issues
#### Pre-filters

In [18]:
iss = pd.read_sql(sql="SELECT value as issues, count FROM pre_filter_value_counts WHERE attribute = 'issues';", con=conn_occ)
pd.set_option('display.max_colwidth', 80)
print(iss)

                                   issues  count
0                      COORDINATE_ROUNDED   2731
1            GEODETIC_DATUM_ASSUMED_WGS84   2926
2                  TAXON_MATCH_HIGHERRANK     62
3        COUNTRY_DERIVED_FROM_COORDINATES      3
4   COORDINATE_UNCERTAINTY_METERS_INVALID     54
5                       TAXON_MATCH_FUZZY      2
6                  COORDINATE_REPROJECTED    213
7                     TYPE_STATUS_INVALID     15
8                  GEODETIC_DATUM_INVALID    165
9              PRESUMED_NEGATED_LONGITUDE     12
10               INDIVIDUAL_COUNT_INVALID      3


#### Post-filters

In [19]:
sql = "SELECT issue, COUNT(occ_id) as count FROM occurrences GROUP BY issue;"
print(pd.read_sql(sql=sql, con=conn_occ))

DatabaseError: Execution failed on sql 'SELECT issue, COUNT(occ_id) as count FROM occurrences GROUP BY issue;': no such column: issue

# Descriptions of filtered records

### Locations

In [ ]:
shp1 = {'file': '{0}{1}_polygons'.format(outDir, summary_name), 'column': None,
        'alias': 'Occurrence records', 'drawbounds': True, 'linewidth': .75, 'linecolor': 'k',
        'fillcolor': None, 'marker':'o'}

# Display occurrence polygons
map_these=[shp1]
    
title="{1} ({0})".format(years, common_name)
functions.MapShapefilePolygons(map_these=map_these, title=title)

### Years represented

In [ ]:
occ_years = [int(x[0]) for x in curs_occ.execute("SELECT strftime('%Y', occurrenceDate) FROM occurrences").fetchall()]
years = connjup.execute("SELECT years_range FROM gbif_requests WHERE request_id = '{0}'".format(gbif_req_id)).fetchone()[0]
years = years.split(',')
yearsrng = list(range(int(years[0]), int(years[1]), 1))
binsnum = int(years[1]) - int(years[0])
plt.hist(occ_years, bins=binsnum)
plt.ylabel("number of occurrences")
plt.xlabel("year")
plt.xticks(yearsrng, rotation=90)
plt.title("Occurrences per Year")

### Months represented

In [ ]:
occ_months = [int(x[0]) for x in curs_occ.execute("SELECT strftime('%m', occurrenceDate) FROM occurrences").fetchall()]
plt.hist(occ_months, bins=range(1, 14), color="g")
plt.ylabel("number of occurrences")
plt.xlabel("month")
plt.xticks(range(1, 13))
plt.title("Occurrences per Month")

### Distribution of coordinate uncertainty values

In [ ]:
occ_cert = [int(x[0]) for x in curs_occ.execute("SELECT coordinateUncertaintyInMeters FROM occurrences").fetchall()]
maxi = np.max(occ_cert)
plt.figure(figsize=(16,4))
plt.hist(occ_cert, bins=50, color="r")
plt.xticks(range(0, maxi, int(maxi/50)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("coordinate uncertainty")
plt.title("Coordinate Uncertainty")

In [ ]:
rng_max = 2000
occ_cert2 = [x for x in occ_cert if x <= rng_max]
plt.figure(figsize=(12,4))
plt.hist(occ_cert2, bins=30, color="m", align='mid')
plt.xticks(range(0, rng_max + 100, int(rng_max/30.)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("coordinate uncertainty")
plt.title("Coordinate Uncertainty")

### Establishment means reported

In [ ]:
establishment = curs_occ.execute("SELECT vals FROM unique_values WHERE field = 'establishment' AND step = 'filter';").fetchall()[0]
for est in establishment:
    est = est.replace('[', '').strip().replace(']', '').replace("'", "")
    print(est)

### Identification qualifiers included

In [ ]:
quali = curs_occ.execute("SELECT DISTINCT vals FROM unique_values WHERE field = 'IDqualifier' AND step = 'filter';").fetchall()[0]
for q in quali:
    q = q.replace('[', '').strip().replace(']', '').replace("'", "")
    print(q)

### Remarks included

In [ ]:
remarks = curs_occ.execute("SELECT DISTINCT remarks FROM occurrences;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

### Attributes returned for the records in the request (pre-filters)

In [ ]:
fields_summary = pd.read_sql("SELECT * FROM gbif_fields_returned", conn_occ)#, index_col='index')
fields_summary.index.name = 'Field'
pd.set_option('display.max_rows', 250)
print(fields_summary)

### Runtime

In [ ]:
t2 = datetime.now()
print(t2 - t1)